In [83]:
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [72]:
degreesTPB_df = pd.read_csv("degrees-that-pay-back.csv")
degreesTPB_df.head(5)

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Accounting,"$46,000.00","$77,100.00",67.6,"$42,200.00","$56,100.00","$108,000.00","$152,000.00"
1,Aerospace Engineering,"$57,700.00","$101,000.00",75.0,"$64,300.00","$82,100.00","$127,000.00","$161,000.00"
2,Agriculture,"$42,600.00","$71,900.00",68.8,"$36,300.00","$52,100.00","$96,300.00","$150,000.00"
3,Anthropology,"$36,800.00","$61,500.00",67.1,"$33,800.00","$45,500.00","$89,300.00","$138,000.00"
4,Architecture,"$41,600.00","$76,800.00",84.6,"$50,600.00","$62,200.00","$97,000.00","$136,000.00"


In [71]:
degreesTPB_df = degreesTPB_df.rename(columns={"Undergraduate Major": "Majors"})
degreesTPB_df.head(5)

,Majors,Starting Median Salary,Mid-Career Median Salary,Major_id
0,Accounting,46000.0,77100.0,1
1,Aerospace Engineering,57700.0,101000.0,2
2,Agriculture,42600.0,71900.0,3
3,Anthropology,36800.0,61500.0,4
4,Architecture,41600.0,76800.0,5


In [54]:
for col in degreesTPB_df.columns:
    if 'Salary' in col:
        degreesTPB_df[col] = degreesTPB_df[col].replace( '[\$,)]','', regex=True ).astype(float)

In [73]:
degreesTPB_df = degreesTPB_df.drop(columns=['Percent change from Starting to Mid-Career Salary', 'Mid-Career 10th Percentile Salary', 'Mid-Career 25th Percentile Salary', 'Mid-Career 75th Percentile Salary', 'Mid-Career 90th Percentile Salary'])
degreesTPB_df.head(5)

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary
0,Accounting,"$46,000.00","$77,100.00"
1,Aerospace Engineering,"$57,700.00","$101,000.00"
2,Agriculture,"$42,600.00","$71,900.00"
3,Anthropology,"$36,800.00","$61,500.00"
4,Architecture,"$41,600.00","$76,800.00"


In [74]:
degreesTPB_df['Major_id'] = degreesTPB_df.index + 1
degreesTPB_df.set_index('Major_id').head(5)

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary
Major_id,,,
1,Accounting,"$46,000.00","$77,100.00"
2,Aerospace Engineering,"$57,700.00","$101,000.00"
3,Agriculture,"$42,600.00","$71,900.00"
4,Anthropology,"$36,800.00","$61,500.00"
5,Architecture,"$41,600.00","$76,800.00"


# Connect to local database

In [57]:
rds_connection_string = "postgres:postgres@localhost:5432/HigherEducation"
engine = create_engine(f'postgresql://{rds_connection_string}')

# Check For Tables

In [58]:
engine.table_names()

['school majors', 'salaries_per_major']

# Use pandas to load csv converted DataFrame into database

In [59]:
degreesTPB_df.head(2)

,Majors,Starting Median Salary,Mid-Career Median Salary,Major_id
0,Accounting,46000.0,77100.0,1
1,Aerospace Engineering,57700.0,101000.0,2


In [75]:
if_exists_param = 'replace'

degreesTPB_df.to_sql(name='salaries_per_major', con=engine, if_exists=if_exists_param, index=False)
degreesTPB_df['Major_id'].to_sql(name='school_majors', con=engine, if_exists=if_exists_param, index=False)


In [84]:
inspector = inspect(engine)
inspector.get_table_names()

['salaries_per_major', 'school_majors']

In [86]:
columns = inspector.get_columns('school_majors')
for c in columns:
    print(c['name'], c["type"])

Major_id BIGINT


# Confirm data has been added by querying the HigherEducation table

In [77]:
pd.read_sql_query('select * from salaries_per_major', con=engine).head(2)

,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Major_id
0,Accounting,"$46,000.00","$77,100.00",1
1,Aerospace Engineering,"$57,700.00","$101,000.00",2


In [78]:
pd.read_sql_query('select * from school_majors', con=engine).head(2)

,Major_id
0,1
1,2


# Perform Join on Primary Key

In [92]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [89]:
# create a session
session = Session(engine)

In [91]:
session.query(salaries_per_major.Major_id, school_majors.Major_id).limit(40).all()

SyntaxError: invalid syntax (<ipython-input-91-2e955ccb649d>, line 1)